In [2]:
#Import all libarys(except ML) 
import pandas as pd                                                             
from google.colab import files
import io
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np


In [3]:
#Upload file into colab and make them .csv
uploaded_train = files.upload()
uploaded_test = files.upload()

Saving train.csv to train.csv


Saving test.csv to test.csv


In [36]:
#Refactor name into they tittle. 
def make_group(name):                                                           
  group_name = re.findall(r'[A-Z][a-z]+\.', name)[0] 
  title_list=['Mr.', 'Miss.', 'Mrs.', 'Master.']
  return group_name if group_name in title_list else 'Other'


#Put median depends on previous group 
def fill_na_age(df):                                                             
  df = df.merge(median_df,on='Name')
  df = pd.get_dummies(df,columns=['Name'],drop_first=True)
  df.Age = df.fillna(0).apply(lambda x:round(x.Median) if x.Age == 0 else round(x.Age), 
                              axis=1)
  df = df.drop(columns=['Median'])
  return df


#Some preparation and Make equallity our Train and Test dataFrame
def make_eq(df):
  df.Name= df.Name.apply(lambda x: make_group(x))
  
  #Make 1 seriers of Sex passengers.0=male,1=female
  df.Sex = pd.get_dummies(df.Sex,drop_first = True)
  #Make 2 series of Pclass.
  df = pd.get_dummies(df,columns=['Pclass'],drop_first = True)

  df.Cabin = df.Cabin.str.get(0) 
  cabin_list = ['A', 'B', 'C', 'D', 'E']
  #Z is a NA_cabin,Y if we have info about cabin 
  df.Cabin = df.fillna('Z').apply(lambda x:x.Cabin if x.Cabin in cabin_list else 'Z', 
                                  axis=1) 
  #Split our 6 type of cabin to different series
  df = pd.get_dummies(df,columns=['Cabin'],drop_first=True) 
   
  #We have 1 empty Fare passenger. Fill them Median of all dataset           
  df.Fare = df.Fare.fillna(45)                  
 
  #Create series - if Passenger travel alone =0 else 1
  df.Parch = df.apply(lambda x:1 if x.Parch+x.SibSp==0 else 0,axis=1)           
  df = df.rename(columns={"Parch": "isAlone"}) 

  #Drop useless columns                                 
  df = df.drop(columns=['Ticket','Embarked','SibSp','Cabin'])                           
  return df


In [37]:
#Make df equallity
train_df = make_eq(pd.read_csv(io.BytesIO(uploaded_train['train.csv'])))        
test_df = make_eq(pd.read_csv(io.BytesIO(uploaded_test['test.csv'])))  

#Create Df(Train+Test) to fill NA Ages()         
#I use median of ALL dataset,because we have not much data.1309 of all passanger and 419 in test dataFrame. 
#So using median of all passenger will be more usefull. That's why we merg two dataset into 1
merg_df = pd.read_csv(io.BytesIO(uploaded_train['train.csv'])).append(         
    pd.read_csv(io.BytesIO(uploaded_test['test.csv'])),ignore_index=True) 
merg_df.Name= merg_df.Name.apply(lambda x:make_group(x))                        
median_df = merg_df.groupby(by='Name',as_index=False)['Age'].median()
median_df.columns = ['Name','Median']

#Fill empty space's on Age Series in dataset
train_df =fill_na_age(train_df)                                                 
test_df = fill_na_age(test_df)                                                  

In [39]:
#Create Target('Survived')
target = train_df.Survived   
#Create Train(Model train by that DF)                                                  
X_train = train_df.drop(columns=['Survived','PassengerId']) 
#Save PassengerId to connect with Predict 'Survived' Series.                    
test_id = test_df.PassengerId                
#Create Test(Model will predict this passenger's)                                   
X_test = test_df.drop(columns=['PassengerId'])        
X_train

,Sex,Age,isAlone,Fare,Pclass_2,Pclass_3,Name_Miss.,Name_Mr.,Name_Mrs.,Name_Other
0,1,22,0,7.2500,0,1,0,1,0,0
1,1,35,1,8.0500,0,1,0,1,0,0
2,1,29,1,8.4583,0,1,0,1,0,0
3,1,54,1,51.8625,0,0,0,1,0,0
4,1,20,1,8.0500,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
886,1,44,1,39.6000,0,0,0,0,0,1
887,0,49,1,25.9292,0,0,0,0,0,1
888,1,38,1,0.0000,0,0,0,0,0,1
889,1,28,0,33.0000,1,0,0,0,0,1


In [43]:
from sklearn.model_selection import GridSearchCV , StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#Tunning hyperparams
#Use SfK,because we need to save balance of classes(People,who died>People,who survived)
# cv=StratifiedKFold(n_splits=3,shuffle=True,random_state=5)                      
# param_test = {                                                                
#  'bootstrap': [True,False],
#  'class_weight':['balanced', 'balanced_subsample'],
#  'max_depth': range(2,10,1),
#  'criterion': ['gini','entropy'],
#  'max_features': ['auto','log2'],
#  'min_samples_leaf': [1, 2, 4,7],
#  'min_samples_split': [2, 3, 5,7],
#  'n_estimators': range(20,140,20)
#               }
# gsearch = GridSearchCV(estimator= RandomForestClassifier(),               
#  param_grid = param_test,verbose=4,n_jobs=-1,cv=cv).fit(X_train,target)
#Print best params,that we found over all candidates, Score(average over validation) and One-time-score 
#print(gsearch.best_params_, gsearch.best_score_,accuracy_score(target, gsearch.predict(X_train))) 

#Use best param of gridsearch 
model=RandomForestClassifier(bootstrap=False, max_depth= 6, max_features= 'auto',
 min_samples_leaf= 2, min_samples_split= 5, n_estimators= 100).fit(X_train,target)
print(accuracy_score(target, model.predict(X_train))) 
                       

0.877665544332211


In [41]:
#Predict Test dataset
answer = model.predict(X_test)   

#Connect our Predictions with PassengersId
solution = pd.concat([pd.Series(answer),test_id,],axis=1)                       
solution = solution.sort_values(by=['PassengerId'])
solution = solution.reset_index(drop=True)
solution = solution[['PassengerId',0]]
solution.columns=['PassengerId','Survived']
solution

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [42]:
#Download Answer to Google Drive
from google.colab import drive                                                  
drive.mount('drive')
solution.to_csv('Answer3.csv',index=False)
!cp Answer3.csv 'drive/My Drive/rep/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
